#### Tried to connect with Postgres

In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
# defining the path
claims = "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_claims"
patient = "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_patient"
provider = "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_provider"
departments = "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_department"
transactions = "abfss://gold@hpadlsacc.dfs.core.windows.net/fact_transactions"
encounter = "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_encounter"

#### error while saving it in the postgres database

In [0]:
tables = ["dim_claims", "dim_patient", "dim_provider", "dim_departments" ,"fact_transactions", "dim_encounter"]

for table in tables:

    df = spark.read.format("delta").load(f"abfss://gold@hpadlsacc.dfs.core.windows.net/{table}")

    df.write \
      .format("jdbc") \
      .option("url", "jdbc:postgresql://postgreshcdb.postgres.database.azure.com:5432/postgres") \
      .option("dbtable", table) \
      .option("user", "vishak@postgreshcdb") \
      .option("password", "@admin001") \
      .option("sslmode", "require")\
      .mode("overwrite") \
      .save()
